In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Blues, BuGn, viridis, Paired, plasma
from bokeh.models import FixedTicker, FactorRange,Color,CustomJS, HoverTool,CategoricalAxis, LabelSet, Label, ColumnDataSource, widgets,CategoricalColorMapper,LinearInterpolator, LinearColorMapper, LogColorMapper
from bokeh.models import (GMapPlot, GMapOptions, Range1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool,  ResetTool, ZoomInTool, ZoomOutTool)
from bokeh.layouts import row, column, widgetbox
from bokeh.models.glyphs import Patches, Line, Circle
from bokeh.charts import Histogram, output_file, show, Bar, color,  Scatter
from bokeh.resources import CDN
import sklearn
from sklearn.utils.validation import check_array
import random
import squarify
import pandas as pd
import numpy as np


output_notebook()


%load_ext sql
%sql sqlite:///mit-poster.db
%sql ATTACH '../jupyterdemo/claims.db' as mem;


In [ ]:
import AlgorexCore as rex

In [ ]:
from hcc import *


In [ ]:
daniel

In [ ]:
yr1_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2008%';
yr2_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2009%';

In [ ]:
jane = Beneficiary(hicno='0220F11E0B2EC004', sex='female', dob='19480601')
jane_alt = Beneficiary(hicno='0220F11E0B2EC004', sex='male', dob='19480601')

#antonio.add_diagnosis(Diagnosis(antonio,"49320",ICDType.NINE))


for icd9 in yr1_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane,code, int(ICDType.NINE)))

for icd9 in yr1_diags + yr2_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane_alt,code, int(ICDType.NINE)))


            

In [ ]:
beneficiary_has_hcc(jane, X)

print(score(jane,"community",Score), score(jane_alt,"community",Score))

In [ ]:
score(jane,"community",Score)[0][0]

In [ ]:
cvars= community_regression()
indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)

In [ ]:
indicator(jane_alt,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)

In [ ]:
len(indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef))



In [ ]:
x = 0
y = 0
width = 40
height = 40
jane_coefs = indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
jane_coefs = sorted(jane_coefs,key= lambda val:  val[1], reverse=True)
print(jane_coefs)
normed = squarify.normalize_sizes([value for name, value in jane_coefs], width, height)
rects = squarify.squarify(normed, x, y, width, height)
shapes = []
annotations = []
counter = 0
for r in rects:
    name, value = jane_coefs[counter]
    shapes.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter = counter + 1
 


r_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes)))


q = figure(x_range=Range1d(x, width), y_range=Range1d(y, height))
q.rect( x='x0', y='y0', width='width', height='height', fill_color='color',
           line_alpha=0, source=r_data)
q.text(x='x1', y='y1', text='text', source=r_data, x_offset=3, y_offset=1
       , text_font_size='5pt')
q.xaxis.visible = False
q.yaxis.visible = False





In [ ]:

janealt_coefs = indicator(jane_alt,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
janealt_coefs = sorted(janealt_coefs,key= lambda val:  val[1], reverse=True)
normed_alt = squarify.normalize_sizes([value for name, value in janealt_coefs], width, height)
rects_alt = squarify.squarify(normed_alt, x, y, width, height)
shapes_alt = []

counter_alt = 0
for r in rects_alt:
    name, value = janealt_coefs[counter]
    shapes_alt.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter_alt = counter_alt + 1

z_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes_alt)))
z_data.data['name'] = [rex.hcc_labels[label] for label in z_data.data['name']]

z = figure(x_range=Range1d(x, width), y_range=Range1d(y, height))
z.rect( x='x0', y='y0', width='width', height='height', fill_color='color',
           line_alpha=0, source=z_data)
z.text(x='x1', y='y1', text='text', source=z_data, x_offset=3, y_offset=1
       , text_font_size='5pt')
z.xaxis.visible = False
z.yaxis.visible = False






In [ ]:
show(row(q,z))

In [ ]:
cohort_study = %sql \
    SELECT d.PatientID, p.SEX, p.DOB \
        from diagnoses d join mem.patients p\
                on d.PatientID = p.PATIENT_ID\
        group by PatientID,p.SEX, p.DOB\
        having COUNT(*) > 20\
        order by Count(*) desc\
        limit 1000;

In [ ]:
def gender(l):
    if l == 'M':
        return 'male'
    else:
        return 'female'


cohort_patients = [Beneficiary(hicno=row[0], sex=gender(row[1]), dob=str(row[2])) for row in cohort_study[1:]]


In [ ]:
hicno = [pat.hicno for pat in cohort_patients]
hicno = tuple(hicno)
diags = %sql SELECT DISTINCT PatientID, Diagnosis from diagnoses where PatientID in $hicno
diags = diags.DataFrame().set_index('PatientId')

for pat in cohort_patients:
    pat_diags = diags['Diagnosis'].loc[pat.hicno].tolist()
    for code in pat_diags:
        pat.add_diagnosis(Diagnosis(pat,code,ICDType.NINE))
        

#jane.add_diagnosis(Diagnosis(jane,code, int(ICDType.NINE)))

In [ ]:
cohort_score  = [score(pat,"community",Score)[0][0] for pat in cohort_patients[0:10]]


In [ ]:
np.mean(cohort_score)

In [ ]:
#indicator(X,CC) & X.in_([cohort_patients[0:10]])  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)